In [20]:
import json

# Load the Yosys-generated JSON netlist
with open('synthesized_XOR.json', 'r') as f:
    netlist = json.load(f)

# Specify the top module manually (adjust the name as needed)
top_module_name = 'xor_chain'
if top_module_name not in netlist.get('modules', {}):
    raise KeyError(f"Module {top_module_name} not found in the netlist.")

module = netlist['modules'][top_module_name]

# Build a mapping for port nodes where numbering restarts for each port.
# For each port, we assign new names as f"{port_name}{index}" where index restarts at 0.
port_node_mapping = {}
for port_name, port in module.get("ports", {}).items():
    bits = port.get("bits", [])
    for idx, bit in enumerate(bits):
        new_node_name = f"{port_name}{idx}"  # numbering restarts for each port
        port_node_mapping[str(bit)] = new_node_name

print("Port node mapping:", port_node_mapping)

# Prepare a circuit dictionary with a set for nodes and a list for gates.
circuit = {"nodes": set(), "gates": []}

# Define a mapping from Yosys cell names to the gate types you want.
cell_mapping = {
    "$_AND_": "AND",
    "$_OR_": "OR",
    "$_NOT_": "NOT",
    "$_ANDNOT_": "ANDNOT",
    "$_ORNOT_": "ORNOT",
    "$_NAND_": "NAND",
    "$_NOR_": "NOR",
    "$_XOR_": "XOR",
    "$_XNOR_": "XNOR",
    # Add additional mappings if desired.
}

# Iterate over each cell in the top module.
for cell_name, cell in module.get("cells", {}).items():
    yosys_type = cell["type"]
    gate_type = cell_mapping.get(yosys_type, yosys_type)
    
    gate_nodes = []
    for port, nets in cell.get("connections", {}).items():
        # In Yosys JSON, nets is usually a list (even for a single connection)
        if isinstance(nets, list):
            net = nets[0]
        else:
            net = nets
        net_str = str(net)
        # Use the new name from our port mapping if it exists, otherwise use the raw net.
        node_name = port_node_mapping.get(net_str, net_str)
        gate_nodes.append(node_name)
        circuit["nodes"].add(node_name)
    
    circuit["gates"].append({"type": gate_type, "nodes": gate_nodes})

# Convert the nodes set to a sorted list for reproducibility.
circuit["nodes"] = sorted(list(circuit["nodes"]))

# Write the resulting circuit dictionary to a JSON file.
with open('circuit.json', 'w') as outfile:
    json.dump(circuit, outfile, indent=4)

print("Circuit has been written to circuit.json")


Port node mapping: {'2': 'A0', '3': 'B0', '4': 'D0', '5': 'F0', '6': 'H0', '7': 'J0', '8': 'L0', '9': 'N0', '10': 'P0', '11': 'C0', '12': 'E0', '13': 'G0', '14': 'I0', '15': 'K0', '16': 'M0', '17': 'O0', '18': 'Q0'}
Circuit has been written to circuit.json


In [21]:
import Scripts.CustomScripts as CS
import Scripts.generate_weight_from_logic as GWL
import json

with open("circuit.json", 'r') as infile:
    circuit = json.load(infile)
print(circuit)
# Preprocess the circuit to ensure that XOR and XNOR gates have the expected number of nodes.
for gate in circuit["gates"]:
    gate_type = gate["type"]
    # Set expected number of nodes based on gate type.
    if gate_type == "XOR":
        expected_nodes = 4  # 3 provided + 1 hidden
    elif gate_type == "XNOR":
        expected_nodes = 5  # 3 provided + 2 hidden
    else:
        continue  # Skip other gate types

    gate_index = circuit["gates"].index(gate)

    # Ensure XOR has a single hidden node appended
    if gate_type == "XOR" and len(gate["nodes"]) == 3:
        # Ensure XOR has a single hidden node appended if it doesn't already have one.
        if len(gate["nodes"]) == 3:
            hidden_node = f"hidden_XOR_{gate_index}_3"
            # Append the hidden node so that the gate becomes [A, B, C, H]
            gate["nodes"].append(hidden_node)
            # Instead of simply appending, insert the hidden node into the circuit's overall node list
            # immediately after the XOR output node.
            output_node = gate["nodes"][2]  # The output node (C)
            if output_node in circuit["nodes"]:
                idx = circuit["nodes"].index(output_node)
                # Insert the hidden node right after the output node, if it isn't already there.
                if idx + 1 >= len(circuit["nodes"]) or circuit["nodes"][idx + 1] != hidden_node:
                    circuit["nodes"].insert(idx + 1, hidden_node)
            else:
                circuit["nodes"].append(hidden_node)
    # Ensure XNOR has exactly 5 nodes and in correct order
    elif gate_type == "XNOR":
        if len(gate["nodes"]) == 3:  # Initial state: ['A', 'B', 'C']
            # Generate hidden nodes
            hidden_0 = f"hidden_XNOR_{gate_index}_H0"
            hidden_1 = f"hidden_XNOR_{gate_index}_H1"
            
            # Insert H0 at position 2 (before C)
            gate["nodes"].insert(2, hidden_0)
            # Append H1 at the end
            gate["nodes"].append(hidden_1)

            # Ensure the hidden nodes are registered in the circuit's node list
            for hidden_node in [hidden_0, hidden_1]:
                if hidden_node not in circuit["nodes"]:
                    circuit["nodes"].append(hidden_node)

print(circuit)
h_total, J_total, nodes = GWL.generate_circuit_weights(circuit,SaveCircuit=True,CircuitName="XOR_chain")

{'nodes': ['A0', 'B0', 'C0', 'D0', 'E0', 'F0', 'G0', 'H0', 'I0', 'J0', 'K0', 'L0', 'M0', 'N0', 'O0', 'P0', 'Q0'], 'gates': [{'type': 'XOR', 'nodes': ['B0', 'A0', 'C0']}, {'type': 'XOR', 'nodes': ['D0', 'C0', 'E0']}, {'type': 'XOR', 'nodes': ['F0', 'E0', 'G0']}, {'type': 'XOR', 'nodes': ['H0', 'G0', 'I0']}, {'type': 'XOR', 'nodes': ['J0', 'I0', 'K0']}, {'type': 'XOR', 'nodes': ['L0', 'K0', 'M0']}, {'type': 'XOR', 'nodes': ['N0', 'M0', 'O0']}, {'type': 'XOR', 'nodes': ['P0', 'O0', 'Q0']}]}
{'nodes': ['A0', 'B0', 'C0', 'hidden_XOR_0_3', 'D0', 'E0', 'hidden_XOR_1_3', 'F0', 'G0', 'hidden_XOR_2_3', 'H0', 'I0', 'hidden_XOR_3_3', 'J0', 'K0', 'hidden_XOR_4_3', 'L0', 'M0', 'hidden_XOR_5_3', 'N0', 'O0', 'hidden_XOR_6_3', 'P0', 'Q0', 'hidden_XOR_7_3'], 'gates': [{'type': 'XOR', 'nodes': ['B0', 'A0', 'C0', 'hidden_XOR_0_3']}, {'type': 'XOR', 'nodes': ['D0', 'C0', 'E0', 'hidden_XOR_1_3']}, {'type': 'XOR', 'nodes': ['F0', 'E0', 'G0', 'hidden_XOR_2_3']}, {'type': 'XOR', 'nodes': ['H0', 'G0', 'I0', 'hi

In [16]:
idx = 36
print(circuit["nodes"][idx])
print(nodes[idx])
print(h_total[idx])

hidden_XOR_6_3
hidden_XOR_6_3
2.0


In [5]:
node_to_gates = {}
for gate in circuit["gates"]:
    for node in gate["nodes"]:
        node_to_gates.setdefault(node, []).append(gate["type"])
for node, types in node_to_gates.items():
    print(f"{node}: {types}")


b1: ['AND', 'AND', 'NAND', 'AND']
a0: ['AND', 'AND', 'AND']
14: ['AND', 'XOR']
b0: ['AND', 'AND', 'AND']
a1: ['AND', 'AND', 'NAND', 'AND']
15: ['AND', 'XOR', 'NAND']
16: ['AND', 'AND', 'XOR', 'NAND']
17: ['NAND', 'AND']
s0: ['AND', 'AND']
18: ['AND', 'AND', 'XOR']
s1: ['XOR']
hidden_XOR_6_3: ['XOR']
a2: ['AND', 'AND', 'AND']
19: ['AND', 'XOR']
20: ['AND', 'NAND', 'XOR']
21: ['NAND', 'NAND']
22: ['XOR', 'NAND', 'XOR']
hidden_XOR_10_3: ['XOR']
b2: ['AND', 'AND', 'AND']
23: ['AND', 'NAND', 'XOR']
24: ['NAND', 'NAND']
25: ['XOR', 'AND', 'XOR']
hidden_XOR_13_3: ['XOR']
26: ['AND', 'NAND', 'XOR']
27: ['NAND', 'NAND', 'XOR']
28: ['AND', 'XOR']
29: ['AND', 'NAND', 'AND']
30: ['NAND', 'NAND']
31: ['XOR', 'NAND', 'XOR']
hidden_XOR_19_3: ['XOR']
32: ['NAND', 'NAND']
33: ['XOR', 'NAND', 'XOR']
hidden_XOR_21_3: ['XOR']
34: ['NAND', 'NAND']
s3: ['XOR']
hidden_XOR_23_3: ['XOR']
35: ['NAND', 'NAND', 'XOR']
36: ['AND', 'NAND', 'XOR']
37: ['NAND', 'NAND']
s4: ['XOR']
hidden_XOR_27_3: ['XOR']
s2: ['XOR']

In [4]:
import json

# Create a dictionary mapping each node name to its index
node_to_index = {node: idx for idx, node in enumerate(nodes)}

# Load the circuit JSON from a file
with open("circuit.json", "r") as infile:
    circuit = json.load(infile)

# For each gate, map its node names to indices
gate_index_map = []
for gate in circuit["gates"]:
    gate_type = gate["type"]
    mapped_nodes = []
    for node in gate["nodes"]:
        # Look up the node index using the mapping dictionary
        index = node_to_index.get(node)
        if index is None:
            print(f"Warning: Node '{node}' not found in node_to_index mapping.")
        mapped_nodes.append(index)
    gate_index_map.append({
        "gate_type": gate_type,
        "node_indices": mapped_nodes
    })

# Print the mapping in a formatted JSON style
print(json.dumps(gate_index_map, indent=4))


[
    {
        "gate_type": "AND",
        "node_indices": [
            8,
            0,
            16
        ]
    },
    {
        "gate_type": "AND",
        "node_indices": [
            1,
            8,
            24
        ]
    },
    {
        "gate_type": "AND",
        "node_indices": [
            9,
            0,
            25
        ]
    },
    {
        "gate_type": "XOR",
        "node_indices": [
            25,
            24,
            17
        ]
    },
    {
        "gate_type": "NAND",
        "node_indices": [
            2,
            8,
            26
        ]
    },
    {
        "gate_type": "NAND",
        "node_indices": [
            9,
            1,
            27
        ]
    },
    {
        "gate_type": "XNOR",
        "node_indices": [
            27,
            26,
            28
        ]
    },
    {
        "gate_type": "NAND",
        "node_indices": [
            10,
            0,
            29
        ]
    },
    {
       